# 드림부스

[드림부스](https://arxiv.org/abs/2208.12242)는 피사체의 이미지 몇 장(3~5장)만으로 스테이블 디퓨전과 같은 텍스트-이미지 모델을 개인화하는 방법입니다. 이를 통해 모델은 다양한 장면, 포즈 및 뷰에서 피사체의 맥락화된 이미지를 생성할 수 있습니다.

![프로젝트 블로그의 드림부스 예시](https://dreambooth.github.io/DreamBooth_files/teaser_static.jpg)
<small>프로젝트의 <a href="https://dreambooth.github.io">블로그</a>에서 가져온 드림부스 예시.</small>

이 가이드에서는 다양한 GPU 크기에 대해 [`CompVis/stable-diffusion-v1-4`](https://huggingface.co/CompVis/stable-diffusion-v1-4) 모델과 Flax를 사용하여 드림부스를 미세 조정하는 방법을 보여줍니다. 이 가이드에서 사용된 드림부스에 대한 모든 학습 스크립트는 [여기](https://github.com/huggingface/diffusers/tree/main/examples/dreambooth)에서 찾을 수 있으며, 작동 방식에 대해 더 자세히 알아보고 싶다면 참조하십시오.

스크립트를 실행하기 전에 라이브러리의 학습 종속성을 설치했는지 확인하십시오. 또한 `main` GitHub 브랜치에서 🧨 Diffusers를 설치하는 것이 좋습니다.

```bash
pip install git+https://github.com/huggingface/diffusers
pip install -U -r diffusers/examples/dreambooth/requirements.txt
```

xFormers는 학습 요구 사항의 일부는 아니지만, 학습 속도를 높이고 메모리 사용량을 줄일 수 있으므로 가능한 경우 [설치](https://huggingface.co/docs/diffusers/main/en/training/../optimization/xformers)하는 것이 좋습니다.

모든 종속성이 설정된 후 다음을 사용하여 [🤗 Accelerate](https://github.com/huggingface/accelerate/) 환경을 초기화합니다.

```bash
accelerate config
```

구성을 선택하지 않고 기본 🤗 Accelerate 환경을 설정하려면 다음을 수행합니다.

```bash
accelerate config default
```

또는 환경이 노트북과 같은 대화형 셸을 지원하지 않는 경우 다음을 사용할 수 있습니다.

In [ ]:
from accelerate.utils import write_basic_config

write_basic_config()

마지막으로, 다음을 사용하여 드림부스로 [개 이미지 몇 장](https://huggingface.co/datasets/diffusers/dog-example)을 다운로드합니다.

In [ ]:
from huggingface_hub import snapshot_download

local_dir = "./dog"
snapshot_download(
    "diffusers/dog-example",
    local_dir=local_dir,
    repo_type="dataset",
    ignore_patterns=".gitattributes",
)

자신의 데이터 세트를 사용하려면 [학습용 데이터 세트 만들기](https://huggingface.co/docs/diffusers/main/en/training/create_dataset) 가이드를 참조하십시오.

## 미세 조정

<Tip warning={true}>

드림부스 미세 조정은 하이퍼파라미터에 매우 민감하며 과적합되기 쉽습니다. 다양한 주제에 대한 권장 설정이 포함된 [심층 분석](https://huggingface.co/blog/dreambooth)을 참조하여 적절한 하이퍼파라미터를 선택하는 데 도움을 받으십시오.

</Tip>

`INSTANCE_DIR` 환경 변수를 개 이미지가 포함된 디렉터리 경로로 설정합니다.

`MODEL_NAME` 환경 변수(Hub 모델 리포지토리 ID 또는 모델 가중치가 포함된 디렉터리 경로)를 지정하고 `pretrained_model_name_or_path` 인수에 전달합니다. `instance_prompt` 인수는 `sks`와 같은 고유 식별자와 이미지가 속한 클래스(이 예에서는 `a photo of sks dog`)를 포함하는 텍스트 프롬프트입니다.

```bash
export MODEL_NAME="CompVis/stable-diffusion-v1-4"
export INSTANCE_DIR="./dog"
export OUTPUT_DIR="path_to_saved_model"
```

그런 다음 다음 명령으로 학습 스크립트(전체 학습 스크립트는 [여기](https://github.com/huggingface/diffusers/blob/main/examples/dreambooth/train_dreambooth.py)에서 찾을 수 있음)를 시작할 수 있습니다.

```bash
accelerate launch train_dreambooth.py \
  --pretrained_model_name_or_path=$MODEL_NAME  \
  --instance_data_dir=$INSTANCE_DIR \
  --output_dir=$OUTPUT_DIR \
  --instance_prompt="a photo of sks dog" \
  --resolution=512 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=1 \
  --learning_rate=5e-6 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --max_train_steps=400 \
  --push_to_hub
```
TPU에 액세스할 수 있거나 더 빠르게 학습하고 싶다면 [Flax 학습 스크립트](https://github.com/huggingface/diffusers/blob/main/examples/dreambooth/train_dreambooth_flax.py)를 사용해 볼 수 있습니다. Flax 학습 스크립트는 그래디언트 체크포인팅 또는 그래디언트 누적을 지원하지 않으므로 최소 30GB 메모리의 GPU가 필요합니다.

스크립트를 실행하기 전에 요구 사항이 설치되어 있는지 확인하십시오.

```bash
pip install -U -r requirements.txt
```

`MODEL_NAME` 환경 변수(Hub 모델 리포지토리 ID 또는 모델 가중치가 포함된 디렉터리 경로)를 지정하고 `pretrained_model_name_or_path` 인수에 전달합니다. `instance_prompt` 인수는 `sks`와 같은 고유 식별자와 이미지가 속한 클래스(이 예에서는 `a photo of sks dog`)를 포함하는 텍스트 프롬프트입니다.

이제 다음 명령으로 학습 스크립트를 시작할 수 있습니다.

```bash
export MODEL_NAME="duongna/stable-diffusion-v1-4-flax"
export INSTANCE_DIR="./dog"
export OUTPUT_DIR="path-to-save-model"

python train_dreambooth_flax.py \
  --pretrained_model_name_or_path=$MODEL_NAME  \
  --instance_data_dir=$INSTANCE_DIR \
  --output_dir=$OUTPUT_DIR \
  --instance_prompt="a photo of sks dog" \
  --resolution=512 \
  --train_batch_size=1 \
  --learning_rate=5e-6 \
  --max_train_steps=400 \
  --push_to_hub
```

## 사전 보존 손실을 사용한 미세 조정

사전 보존은 과적합 및 언어 드리프트를 방지하는 데 사용됩니다(자세한 내용은 [논문](https://arxiv.org/abs/2208.12242) 참조). 사전 보존을 위해 학습 프로세스의 일부로 동일한 클래스의 다른 이미지를 사용합니다. 좋은 점은 Stable Diffusion 모델 자체를 사용하여 해당 이미지를 생성할 수 있다는 것입니다! 학습 스크립트는 생성된 이미지를 지정한 로컬 경로에 저장합니다.

저자는 사전 보존을 위해 `num_epochs * num_samples` 이미지를 생성하는 것을 권장합니다. 대부분의 경우 200-300개의 이미지가 잘 작동합니다.

```bash
export MODEL_NAME="CompVis/stable-diffusion-v1-4"
export INSTANCE_DIR="./dog"
export CLASS_DIR="path_to_class_images"
export OUTPUT_DIR="path_to_saved_model"

accelerate launch train_dreambooth.py \
  --pretrained_model_name_or_path=$MODEL_NAME  \
  --instance_data_dir=$INSTANCE_DIR \
  --class_data_dir=$CLASS_DIR \
  --output_dir=$OUTPUT_DIR \
  --with_prior_preservation --prior_loss_weight=1.0 \
  --instance_prompt="a photo of sks dog" \
  --class_prompt="a photo of dog" \
  --resolution=512 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=1 \
  --learning_rate=5e-6 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --num_class_images=200 \
  --max_train_steps=800 \
  --push_to_hub
```
```bash
export MODEL_NAME="duongna/stable-diffusion-v1-4-flax"
export INSTANCE_DIR="./dog"
export CLASS_DIR="path-to-class-images"
export OUTPUT_DIR="path-to-save-model"

python train_dreambooth_flax.py \
  --pretrained_model_name_or_path=$MODEL_NAME  \
  --instance_data_dir=$INSTANCE_DIR \
  --class_data_dir=$CLASS_DIR \
  --output_dir=$OUTPUT_DIR \
  --with_prior_preservation --prior_loss_weight=1.0 \
  --instance_prompt="a photo of sks dog" \
  --class_prompt="a photo of dog" \
  --resolution=512 \
  --train_batch_size=1 \
  --learning_rate=5e-6 \
  --num_class_images=200 \
  --max_train_steps=800 \
  --push_to_hub
```

## 텍스트 인코더와 UNet 미세 조정

이 스크립트를 사용하면 `unet`과 함께 `text_encoder`를 미세 조정할 수도 있습니다. 실험 결과(자세한 내용은 [🧨 Diffusers를 사용한 DreamBooth로 Stable Diffusion 학습](https://huggingface.co/blog/dreambooth) 게시물 참조), 특히 얼굴 이미지를 생성할 때 훨씬 더 나은 결과를 얻을 수 있습니다.

<Tip warning={true}>

텍스트 인코더를 학습하려면 추가 메모리가 필요하며 16GB GPU에는 맞지 않습니다. 이 옵션을 사용하려면 최소 24GB VRAM이 필요합니다.

</Tip>

`text_encoder`와 `unet`의 미세 조정을 활성화하려면 학습 스크립트에 `--train_text_encoder` 인수를 전달합니다.

```bash
export MODEL_NAME="CompVis/stable-diffusion-v1-4"
export INSTANCE_DIR="./dog"
export CLASS_DIR="path_to_class_images"
export OUTPUT_DIR="path_to_saved_model"

accelerate launch train_dreambooth.py \
  --pretrained_model_name_or_path=$MODEL_NAME  \
  --train_text_encoder \
  --instance_data_dir=$INSTANCE_DIR \
  --class_data_dir=$CLASS_DIR \
  --output_dir=$OUTPUT_DIR \
  --with_prior_preservation --prior_loss_weight=1.0 \
  --instance_prompt="a photo of sks dog" \
  --class_prompt="a photo of dog" \
  --resolution=512 \
  --train_batch_size=1 \
  --use_8bit_adam \
  --gradient_checkpointing \
  --learning_rate=2e-6 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --num_class_images=200 \
  --max_train_steps=800 \
  --push_to_hub
```
```bash
export MODEL_NAME="duongna/stable-diffusion-v1-4-flax"
export INSTANCE_DIR="./dog"
export CLASS_DIR="path-to-class-images"
export OUTPUT_DIR="path-to-save-model"

python train_dreambooth_flax.py \
  --pretrained_model_name_or_path=$MODEL_NAME  \
  --train_text_encoder \
  --instance_data_dir=$INSTANCE_DIR \
  --class_data_dir=$CLASS_DIR \
  --output_dir=$OUTPUT_DIR \
  --with_prior_preservation --prior_loss_weight=1.0 \
  --instance_prompt="a photo of sks dog" \
  --class_prompt="a photo of dog" \
  --resolution=512 \
  --train_batch_size=1 \
  --learning_rate=2e-6 \
  --num_class_images=200 \
  --max_train_steps=800 \
  --push_to_hub
```

## LoRA를 사용한 미세 조정

대규모 언어 모델의 저순위 적응(LoRA)이라는 미세 조정 기법을 사용하여 DreamBooth에서 대규모 모델 학습을 가속화할 수도 있습니다. 자세한 내용은 [LoRA 학습](https://huggingface.co/docs/diffusers/main/en/training/./lora#dreambooth) 가이드를 참조하십시오.

## 학습 중 체크포인트 저장

드림부스로 학습할 때는 과적합되기 쉬우므로 학습 과정에서 정기적으로 체크포인트를 저장하는 것이 유용할 수 있습니다. 중간 체크포인트 중 하나가 최종 모델보다 실제로 더 잘 작동할 수도 있습니다! 학습 스크립트에 다음 인수를 전달하여 체크포인트 저장을 활성화합니다.

```bash
  --checkpointing_steps=500
```

`output_dir`의 하위 폴더에 전체 학습 상태를 저장합니다. 하위 폴더 이름은 접두사 `checkpoint-`로 시작하고 그 뒤에 지금까지 수행된 단계 수가 옵니다. 예를 들어 `checkpoint-1500`은 1500 학습 단계 후에 저장된 체크포인트입니다.

### 저장된 체크포인트에서 학습 재개

저장된 체크포인트에서 학습을 재개하려면 스크립트에 `--resume_from_checkpoint` 인수를 전달하고 사용할 체크포인트 이름을 지정합니다. 특수 문자열 `"latest"`를 사용하여 마지막으로 저장된 체크포인트(단계 수가 가장 많은 체크포인트)에서 재개할 수도 있습니다. 예를 들어 다음은 1500단계 후에 저장된 체크포인트에서 학습을 재개합니다.

```bash
  --resume_from_checkpoint="checkpoint-1500"
```

원하는 경우 하이퍼파라미터를 조정할 수 있는 좋은 기회입니다.

### 저장된 체크포인트에서 추론

저장된 체크포인트는 학습 재개에 적합한 형식으로 저장됩니다. 모델 가중치뿐만 아니라 최적화 프로그램, 데이터 로더 및 학습률의 상태도 포함합니다.

**`"accelerate>=0.16.0"`**이 설치되어 있는 경우 다음 코드를 사용하여 중간 체크포인트에서 추론을 실행합니다.

In [ ]:
from diffusers import DiffusionPipeline, UNet2DConditionModel
from transformers import CLIPTextModel
import torch

# 학습에 사용된 것과 동일한 인수(모델, 리비전)로 파이프라인 로드
model_id = "CompVis/stable-diffusion-v1-4"

unet = UNet2DConditionModel.from_pretrained("/sddata/dreambooth/daruma-v2-1/checkpoint-100/unet")

# `--args.train_text_encoder`로 학습한 경우 텍스트 인코더도 로드해야 합니다.
text_encoder = CLIPTextModel.from_pretrained("/sddata/dreambooth/daruma-v2-1/checkpoint-100/text_encoder")

pipeline = DiffusionPipeline.from_pretrained(model_id, unet=unet, text_encoder=text_encoder, dtype=torch.float16)
pipeline.to("cuda")

# 추론을 수행하거나 저장하거나 허브에 푸시합니다.
pipeline.save_pretrained("dreambooth-pipeline")

**`"accelerate<0.16.0"`**이 설치되어 있는 경우 먼저 추론 파이프라인으로 변환해야 합니다.

In [ ]:
from accelerate import Accelerator
from diffusers import DiffusionPipeline

# 학습에 사용된 것과 동일한 인수(모델, 리비전)로 파이프라인 로드
model_id = "CompVis/stable-diffusion-v1-4"
pipeline = DiffusionPipeline.from_pretrained(model_id)

accelerator = Accelerator()

# 초기 학습에 `--train_text_encoder`가 사용된 경우 text_encoder 사용
unet, text_encoder = accelerator.prepare(pipeline.unet, pipeline.text_encoder)

# 체크포인트 경로에서 상태 복원. 여기서는 절대 경로를 사용해야 합니다.
accelerator.load_state("/sddata/dreambooth/daruma-v2-1/checkpoint-100")

# 래핑되지 않은 모델로 파이프라인 재구성(.unet 및 .text_encoder에 대한 할당도 작동해야 함)
pipeline = DiffusionPipeline.from_pretrained(
    model_id,
    unet=accelerator.unwrap_model(unet),
    text_encoder=accelerator.unwrap_model(text_encoder),
)

# 추론을 수행하거나 저장하거나 허브에 푸시합니다.
pipeline.save_pretrained("dreambooth-pipeline")

## 다양한 GPU 크기에 대한 최적화

하드웨어에 따라 16GB에서 8GB까지의 GPU에서 DreamBooth를 최적화하는 몇 가지 다른 방법이 있습니다!

### xFormers

[xFormers](https://github.com/facebookresearch/xformers)는 트랜스포머 최적화를 위한 도구 상자이며, 🧨 Diffusers에서 사용되는 [메모리 효율적인 어텐션](https://facebookresearch.github.io/xformers/components/ops.html#module-xformers.ops) 메커니즘을 포함합니다. [xFormers를 설치](https://huggingface.co/docs/diffusers/main/en/training/./optimization/xformers)한 다음 학습 스크립트에 다음 인수를 추가해야 합니다.

```bash
  --enable_xformers_memory_efficient_attention
```

xFormers는 Flax에서 사용할 수 없습니다.

### 그라데이션을 없음으로 설정

메모리 사용량을 줄이는 또 다른 방법은 [그래디언트를 0으로 설정하는 대신 `None`으로 설정](https://pytorch.org/docs/stable/generated/torch.optim.Optimizer.zero_grad.html)하는 것입니다. 그러나 이렇게 하면 특정 동작이 변경될 수 있으므로 문제가 발생하면 이 인수를 제거해 보십시오. 학습 스크립트에 다음 인수를 추가하여 그래디언트를 `None`으로 설정합니다.

### 16GB GPU

그래디언트 체크포인팅과 [bitsandbytes](https://github.com/TimDettmers/bitsandbytes) 8비트 옵티마이저의 도움으로 16GB GPU에서 DreamBooth를 학습할 수 있습니다. bitsandbytes가 설치되어 있는지 확인하십시오.

```bash
pip install bitsandbytes
```

그런 다음 `--use_8bit_adam` 옵션을 학습 스크립트에 전달합니다.

### 12GB GPU

12GB GPU에서 DreamBooth를 실행하려면 그래디언트 체크포인팅, 8비트 옵티마이저, xFormers를 활성화하고 그래디언트를 `None`으로 설정해야 합니다.

### 8GB GPU

8GB GPU의 경우 [DeepSpeed](https://www.deepspeed.ai/)의 도움을 받아 VRAM에서 CPU 또는 NVME로 일부 텐서를 오프로드하여 GPU 메모리가 적은 상태에서 학습할 수 있습니다.

다음 명령을 실행하여 🤗 Accelerate 환경을 구성합니다.

```bash
accelerate config
```

구성 중에 DeepSpeed를 사용할 것인지 확인합니다. 이제 DeepSpeed 스테이지 2, fp16 혼합 정밀도, 모델 매개변수 및 옵티마이저 상태를 CPU로 오프로드하는 것을 결합하여 8GB 미만의 VRAM에서 학습할 수 있습니다. 단점은 약 25GB의 더 많은 시스템 RAM이 필요하다는 것입니다. 자세한 구성 옵션은 [DeepSpeed 설명서](https://huggingface.co/docs/accelerate/usage_guides/deepspeed)를 참조하십시오.

또한 상당한 속도 향상을 위해 기본 Adam 옵티마이저를 DeepSpeed의 최적화된 Adam 버전인 [`deepspeed.ops.adam.DeepSpeedCPUAdam`](https://deepspeed.readthedocs.io/en/latest/optimizers.html#adam-cpu)으로 변경해야 합니다. `DeepSpeedCPUAdam`을 활성화하려면 시스템의 CUDA 도구 체인 버전이 PyTorch와 함께 설치된 버전과 동일해야 합니다.

8비트 옵티마이저는 현재 DeepSpeed와 호환되지 않는 것 같습니다.

다음 명령으로 학습을 시작합니다.

## 추론

모델을 학습한 후에는 모델이 저장된 경로를 지정하고 [StableDiffusionPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/stable_diffusion/text2img#diffusers.StableDiffusionPipeline)에서 추론에 사용합니다. 프롬프트에 학습 중에 사용된 특수 `식별자`(`sks`가 이전 예제에서 사용됨)가 포함되어 있는지 확인합니다.

**`"accelerate>=0.16.0"`**이 설치되어 있는 경우 다음 코드를 사용하여 중간 체크포인트에서 추론을 실행할 수 있습니다.

In [ ]:
from diffusers import DiffusionPipeline
import torch

model_id = "path_to_saved_model"
pipe = DiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")

prompt = "A photo of sks dog in a bucket"
image = pipe(prompt, num_inference_steps=50, guidance_scale=7.5).images[0]

image.save("dog-bucket.png")

[저장된 학습 체크포인트](#inference-from-a-saved-checkpoint) 중 하나에서 추론을 실행할 수도 있습니다.

## 만약

lora 및 전체 드림부스 스크립트를 사용하여 텍스트-이미지 [IF 모델](https://huggingface.co/DeepFloyd/IF-I-XL-v1.0) 및 2단계 업스케일러 [IF 모델](https://huggingface.co/DeepFloyd/IF-II-L-v1.0)을 학습할 수 있습니다.

IF에는 예측된 분산이 있으며, 미세 조정 스크립트는 모델 예측 오류만 학습하므로 미세 조정된 IF 모델의 경우 고정 분산 일정으로 전환합니다. 전체 미세 조정 스크립트는 전체 저장된 모델에 대한 스케줄러 구성을 업데이트합니다. 그러나 저장된 LoRA 가중치를 로드할 때는 파이프라인의 스케줄러 구성도 업데이트해야 합니다.

In [ ]:
from diffusers import DiffusionPipeline

pipe = DiffusionPipeline.from_pretrained("DeepFloyd/IF-I-XL-v1.0")

pipe.load_lora_weights("<lora weights path>")

# 스케줄러 구성을 고정 분산 일정으로 업데이트합니다.
pipe.scheduler = pipe.scheduler.__class__.from_config(pipe.scheduler.config, variance_type="fixed_small")

또한 IF에는 몇 가지 대체 CLI 플래그가 필요합니다.

`--resolution=64`: IF는 픽셀 공간 확산 모델입니다. 압축되지 않은 픽셀에서 작동하려면 입력 이미지의 해상도가 훨씬 작아야 합니다.

`--pre_compute_text_embeddings`: IF는 텍스트 인코더로 [T5](https://huggingface.co/docs/transformers/model_doc/t5)를 사용합니다. GPU 메모리를 절약하기 위해 모든 텍스트 임베딩을 미리 계산한 다음 T5를 할당 해제합니다.

`--tokenizer_max_length=77`: T5는 기본 텍스트 길이가 더 길지만 기본 IF 인코딩 절차는 더 작은 숫자를 사용합니다.

`--text_encoder_use_attention_mask`: T5는 어텐션 마스크를 텍스트 인코더에 전달합니다.

### 팁과 요령

모델의 해상도가 낮아 세밀한 디테일을 표현하기 어렵기 때문에 1단계 모델을 미세 조정하는 데 LoRA로 충분하다는 것을 알았습니다.

일반적이거나 시각적으로 복잡하지 않은 개체 개념의 경우 업스케일러를 미세 조정하지 않아도 됩니다. 업스케일러에 전달되는 프롬프트를 조정하여 인스턴스 프롬프트에서 새 토큰을 제거해야 합니다. 즉, 1단계 프롬프트가 "sks 개"인 경우 2단계 프롬프트에 "개"를 사용합니다.

원래 학습 세트에 없는 얼굴과 같은 세밀한 디테일의 경우 2단계 업스케일러를 전체 미세 조정하는 것이 LoRA 미세 조정 2단계보다 낫다는 것을 알았습니다.

얼굴과 같은 세밀한 디테일의 경우 더 큰 배치 크기와 함께 더 낮은 학습률이 가장 효과적이라는 것을 알았습니다.

2단계의 경우 더 낮은 학습률도 필요하다는 것을 알았습니다.

학습 스크립트에 사용된 DPM 솔버 스케줄러보다 기본적으로 더 많은 노이즈 제거 단계가 있는 DDPM 스케줄러가 때때로 더 잘 작동한다는 것을 실험적으로 발견했습니다.

### 2단계 추가 검증 이미지

2단계 검증에는 이미지를 업스케일링해야 하며, 학습 세트의 축소된 버전을 다운로드할 수 있습니다.

In [ ]:
from huggingface_hub import snapshot_download

local_dir = "./dog_downsized"
snapshot_download(
    "diffusers/dog-example-downsized",
    local_dir=local_dir,
    repo_type="dataset",
    ignore_patterns=".gitattributes",
)

### IF 1단계 LoRA 드림부스

이 학습 구성에는 약 28GB의 VRAM이 필요합니다.

```sh
export MODEL_NAME="DeepFloyd/IF-I-XL-v1.0"
export INSTANCE_DIR="dog"
export OUTPUT_DIR="dreambooth_dog_lora"

accelerate launch train_dreambooth_lora.py \
  --report_to wandb \
  --pretrained_model_name_or_path=$MODEL_NAME  \
  --instance_data_dir=$INSTANCE_DIR \
  --output_dir=$OUTPUT_DIR \
  --instance_prompt="a sks dog" \
  --resolution=64 \
  --train_batch_size=4 \
  --gradient_accumulation_steps=1 \
  --learning_rate=5e-6 \
  --scale_lr \
  --max_train_steps=1200 \
  --validation_prompt="a sks dog" \
  --validation_epochs=25 \
  --checkpointing_steps=100 \
  --pre_compute_text_embeddings \
  --tokenizer_max_length=77 \
  --text_encoder_use_attention_mask
```

### IF 2단계 LoRA 드림부스

`--validation_images`: 이러한 이미지는 검증 단계에서 업스케일링됩니다.

`--class_labels_conditioning=timesteps`: 2단계에 필요한 UNet에 추가 조건을 전달합니다.

`--learning_rate=1e-6`: 1단계보다 낮은 학습률입니다.

`--resolution=256`: 업스케일러는 더 높은 해상도의 입력을 예상합니다.

```sh
export MODEL_NAME="DeepFloyd/IF-II-L-v1.0"
export INSTANCE_DIR="dog"
export OUTPUT_DIR="dreambooth_dog_upscale"
export VALIDATION_IMAGES="dog_downsized/image_1.png dog_downsized/image_2.png dog_downsized/image_3.png dog_downsized/image_4.png"

python train_dreambooth_lora.py \
    --report_to wandb \
    --pretrained_model_name_or_path=$MODEL_NAME \
    --instance_data_dir=$INSTANCE_DIR \
    --output_dir=$OUTPUT_DIR \
    --instance_prompt="a sks dog" \
    --resolution=256 \
    --train_batch_size=4 \
    --gradient_accumulation_steps=1 \
    --learning_rate=1e-6 \ 
    --max_train_steps=2000 \
    --validation_prompt="a sks dog" \
    --validation_epochs=100 \
    --checkpointing_steps=500 \
    --pre_compute_text_embeddings \
    --tokenizer_max_length=77 \
    --text_encoder_use_attention_mask \
    --validation_images $VALIDATION_IMAGES \
    --class_labels_conditioning=timesteps
```

### IF 1단계 전체 드림부스

`--skip_save_text_encoder`: 전체 모델을 학습할 때 미세 조정된 모델과 함께 전체 T5를 저장하는 것을 건너뜁니다. 원래 모델에서 로드된 T5로 파이프라인을 계속 로드할 수 있습니다.

`use_8bit_adam`: 옵티마이저 상태의 크기 때문에 전체 XL IF 모델을 8비트 Adam으로 학습하는 것이 좋습니다.

`--learning_rate=1e-7`: 전체 드림부스의 경우 IF는 매우 낮은 학습률이 필요합니다. 학습률이 높을수록 모델 품질이 저하됩니다. 더 큰 배치 크기로 학습률을 높일 수 있다는 점에 유의하십시오.

8비트 Adam과 배치 크기 4를 사용하면 모델을 약 48GB VRAM에서 학습할 수 있습니다.

```sh
export MODEL_NAME="DeepFloyd/IF-I-XL-v1.0"

export INSTANCE_DIR="dog"
export OUTPUT_DIR="dreambooth_if"

accelerate launch train_dreambooth.py \
  --pretrained_model_name_or_path=$MODEL_NAME  \
  --instance_data_dir=$INSTANCE_DIR \
  --output_dir=$OUTPUT_DIR \
  --instance_prompt="a photo of sks dog" \
  --resolution=64 \
  --train_batch_size=4 \
  --gradient_accumulation_steps=1 \
  --learning_rate=1e-7 \
  --max_train_steps=150 \
  --validation_prompt "a photo of sks dog" \
  --validation_steps 25 \
  --text_encoder_use_attention_mask \
  --tokenizer_max_length 77 \
  --pre_compute_text_embeddings \
  --use_8bit_adam \
  --set_grads_to_none \
  --skip_save_text_encoder \
  --push_to_hub
```

### IF 2단계 전체 드림부스

`--learning_rate=5e-6`: 배치 크기가 4로 작을 경우 학습률이 1e-8만큼 낮아야 한다는 것을 발견했습니다.

`--resolution=256`: 업스케일러는 더 높은 해상도의 입력을 예상합니다.

`--train_batch_size=2` 및 `--gradient_accumulation_steps=6`: 특히 얼굴을 포함한 2단계 전체 학습에는 큰 유효 배치 크기가 필요하다는 것을 발견했습니다.

```sh
export MODEL_NAME="DeepFloyd/IF-II-L-v1.0"
export INSTANCE_DIR="dog"
export OUTPUT_DIR="dreambooth_dog_upscale"
export VALIDATION_IMAGES="dog_downsized/image_1.png dog_downsized/image_2.png dog_downsized/image_3.png dog_downsized/image_4.png"

accelerate launch train_dreambooth.py \
  --report_to wandb \
  --pretrained_model_name_or_path=$MODEL_NAME \
  --instance_data_dir=$INSTANCE_DIR \
  --output_dir=$OUTPUT_DIR \
  --instance_prompt="a sks dog" \
  --resolution=256 \
  --train_batch_size=2 \
  --gradient_accumulation_steps=6 \
  --learning_rate=5e-6 \
  --max_train_steps=2000 \
  --validation_prompt="a sks dog" \
  --validation_steps=150 \
  --checkpointing_steps=500 \
  --pre_compute_text_embeddings \
  --tokenizer_max_length=77 \
  --text_encoder_use_attention_mask \
  --validation_images $VALIDATION_IMAGES \
  --class_labels_conditioning timesteps \
  --push_to_hub
```